In [1]:
# Import libraries

import numpy as np
import pickle
import tensorflow as tf
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import layers, regularizers
# from keras_tuner.tuners import RandomSearch
from tensorflow.keras.optimizers import Adam

from tensorflow import keras 
!pip install -q -U keras-tuner 
import keras_tuner 
from keras_tuner import RandomSearch

In [2]:
 # Loading pickles 

def load_color_inputs():
    X_train_left = pickle.load(open('X_train_left.pkl', "rb"))
    X_val_left = pickle.load(open('X_val_left.pkl', "rb"))
    X_test_left = pickle.load(open('X_test_left.pkl', "rb"))
    return X_train_left, X_val_left, X_test_left

X_train_left, X_val_left, X_test_left = load_color_inputs()

def load_outputs():
    Y_train_left = pickle.load(open('Y_train_left.pkl', "rb"))
    Y_val_left = pickle.load(open('Y_val_left.pkl', "rb"))
    Y_test_left = pickle.load(open('Y_test_left.pkl', "rb"))
    return Y_train_left, Y_val_left, Y_test_left

Y_train_left, Y_val_left, Y_test_left = load_outputs()

Y_train_left = np.argmax(Y_train_left, axis=-1)
Y_val_left = np.argmax(Y_val_left, axis=-1)


In [3]:
 print(Y_train_left.shape) 

def build_model(hp):
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())

    
    for i in range(hp.Int('layers', 2, 4)): # Searching through for 2, 3 and 4 hidden layers
        model.add(tf.keras.layers.Dense(
            units=hp.Int('units_' + str(i), 32, 512, step=32), # Searching for optimal amount of nodes per hidden layer from 32 to 512 with step size of 32
            activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid']))) # Searching for optimal activation method
    model.add(tf.keras.layers.Dense(210, activation='softmax')) # Output layer is kept out of the for loop because that is fixed
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                     values=[1e-2, 1e-3, 1e-4])), # Learning Rate of 0.01, 0.001 and 0.0001
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy', # Objective is to maximize validation accuracy
    max_trials=5, # Trial 5 times
    executions_per_trial=3,
    overwrite=False, 
    )

tuner.search_space_summary()

tuner.search(X_train_left, Y_train_left, 
             epochs=50, 
             validation_data=(X_val_left, Y_val_left))

tuner.results_summary()

# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
CNN_left_2 = best_model.build(input_shape=(64, 64, 3))
best_model.summary()

(8820,)
INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json
Search space summary
Default search space size: 10
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 4, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
act_2 (Choice)
{'de

ValueError: Received incompatible tensor with shape (5, 5, 3, 32) when attempting to restore variable with shape (3, 3, 3, 32) and name layer_with_weights-0/kernel/.ATTRIBUTES/VARIABLE_VALUE.

In [4]:
best_model.save('CNN_left_2', save_format='h5') 

from keras.utils import to_categorical 

Y_train_left = to_categorical(Y_train_left, dtype='float32') 
Y_val_left = to_categorical(Y_val_left, dtype='float32') 
Y_test_left = to_categorical(Y_test_left, dtype='float32') 


# Running model on training data 
best_model.compile(optimizer='Adam',
                   loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                   metrics=['accuracy']) 

history = best_model.fit(X_train_left, Y_train_left, epochs=20, batch_size=64, validation_data=(X_val_left, Y_val_left))

# Retrieving loss function 

def summarize_diagnostics(history):
    # loss
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='yellow', label='validation')

    # accuracy
    plt.subplot(212)
    plt.title('Accuracy', y=-0.4)
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='yellow', label='validation')
    plt.legend(['training', 'validation'])
    plt.show()

summarize_diagnostics(history)

NameError: name 'best_model' is not defined

In [4]:
# Importing libraries
import tensorflow as tf
import pickle
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sbn
import itertools


# Loading data
def load_images():
    X_train_left = pickle.load(open('X_train_left.pkl', 'rb'))
    X_val_left = pickle.load(open('X_val_left.pkl', 'rb'))
    X_test_left = pickle.load(open('X_test_left.pkl', 'rb'))
    return X_train_left, X_val_left, X_test_left

X_train_left, X_val_left, X_test_left = load_images()

def load_labels():
    Y_train_left = pickle.load(open('Y_train_left.pkl', 'rb'))
    Y_val_left = pickle.load(open('Y_val_left.pkl', 'rb'))
    Y_test_left = pickle.load(open('Y_test_left.pkl', 'rb'))
    return Y_train_left, Y_val_left, Y_test_left

Y_train_left, Y_val_left, Y_test_left = load_labels()


model = tf.keras.models.load_model('CNN_left_2')


# Loading model

# loss, accuracy = best_model.evaluate(X_test_left, Y_test_left, verbose=0)
# print(loss, accuracy)

# # Confusion Matrices for Validation Data

# # Make label from one-hot encoding for confusion matrices
# Y_train_left = np.argmax(Y_train_left, axis=1)
# Y_val_left = np.argmax(Y_val_left, axis=1)
# Y_test_left = np. argmax(Y_test_left, axis=1)


# predictions_val_left = best_model.predict(X_val_left)
# pred_val_labels_left = np.argmax(predictions_val_left, axis=1)

# CM1_1 = confusion_matrix(Y_val_left, pred_val_labels_left, labels=np.arange(0,70))
# ax = plt.axes()
# sbn.heatmap(CM1_1, annot=False,
#             annot_kws = {"size":10},
#             xticklabels = np.arange(0,70,10),
#             yticklabels = np.arange(0,70,10), ax = ax),
# ax.set_title("CM for Classes 0 - 70 of Validation Data")
# plt.xticks(ticks=np.arange(0,70,10), labels=np.arange(0,70,10))
# plt.yticks(ticks=np.arange(0,70,10), labels=np.arange(0,70,10))
# plt.show()

# CM1_2 = confusion_matrix(Y_val_left, pred_val_labels_left, labels=np.arange(70,140))
# ax = plt.axes()
# sbn.heatmap(CM1_2, annot=False,
#             annot_kws = {"size":10},
#             xticklabels = np.arange(70,140,10),
#             yticklabels = np.arange(70,140,10), ax=ax),
# ax.set_title("CM for Classes 70 - 140 of Validation Data")
# plt.xticks(ticks=np.arange(0,70,10), labels=np.arange(70,140,10))
# plt.yticks(ticks=np.arange(0,70,10), labels=np.arange(70,140,10))
# plt.show()

# CM1_3 = confusion_matrix(Y_val_left, pred_val_labels_left, labels = np.arange(140, 210))
# ax = plt.axes()
# sbn.heatmap(CM1_3, annot=False,
#             annot_kws = {"size":10},
#             xticklabels = np.arange(140,210,10),
#             yticklabels = np.arange(140,210,10), ax = ax),
# ax.set_title("CM for Classes 140 - 210 of Validation Data")
# plt.xticks(ticks=np.arange(0,70,10), labels=np.arange(140,210,10))
# plt.yticks(ticks=np.arange(0,70,10), labels=np.arange(140,210,10))
# plt.show()


# # Confusion Matrix for Test Data

predictions_test_left = best_model.predict(X_test_left)
pred_test_labels_left = np.argmax(predictions_test_left, axis=1)

# CM2_1 = confusion_matrix(Y_test_left, pred_test_labels_left, labels = np.arange(0,70))
# ax = plt.axes()
# sbn.heatmap(CM2_1, annot=False,
#             annot_kws={"size":10},
#             xticklabels=np.arange(0,70,10),
#             yticklabels=np.arange(0,70,10), ax=ax)
# ax.set_title("CM for Classes 0 - 70 of Test Data")
# plt.xticks(ticks=np.arange(0,70,10), labels=np.arange(0,70,10))
# plt.yticks(ticks=np.arange(0,70,10), labels=np.arange(0,70,10))
# plt.show()

# CM2_2 = confusion_matrix(Y_test_left, pred_test_labels_left, labels = np.arange(70,140))
# ax = plt.axes()
# sbn.heatmap(CM2_2, annot=False,
#             annot_kws={"size":10},
#             xticklabels = np.arange(70,140,10),
#             yticklabels = np.arange(70,140,10), ax=ax)
# ax.set_title("CM for Classes 70 - 140 of Test Data")
# plt.xticks(ticks=np.arange(0,70,10), labels=np.arange(70,140,10))
# plt.yticks(ticks=np.arange(0,70,10), labels=np.arange(70,140,10))
# plt.show()

# CM2_3 = confusion_matrix(Y_test_left, pred_test_labels_left, labels = np.arange(140,210))
# ax = plt.axes()
# sbn.heatmap(CM2_3, annot=False,
#             annot_kws={"size":10},
#             xticklabels = np.arange(140,210,10),
#             yticklabels = np.arange(140,210,10), ax = ax)
# ax.set_title("CM for Classes 140 - 210 of Test Data")
# plt.xticks(ticks=np.arange(0,70,10), labels=np.arange(140,210,10))
# plt.yticks(ticks=np.arange(0,70,10), labels=np.arange(140,210,10))
# plt.show()

report_val = classification_report(Y_val_left, pred_val_labels_left)
print(report_val)

# # Classification Report for Test Data

# report_test = classification_report(Y_test_left, pred_test_labels_left, digits=5, output_dict=True)
# print(report_test)

print(pred_labels_left[2730:2745])

2023-01-12 08:59:11.360085: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 36.91MiB (rounded to 38707200)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-01-12 08:59:11.360708: W tensorflow/core/common_runtime/bfc_allocator.cc:491] ***********xx**********xxxxx***********xxx______________***************_____________________________


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.